# Searching for new functional bases for quantum algorithms

### Imports

In [21]:
from lib import *
from typing import Callable


## Function Definitions

In [22]:
def parity(secret: int) -> Callable[[int], int]:
	"""
	This is the function that generates the functions used in Bernstein-Vazirani.
	"""
	def f(x: int) -> int:
		result = 0
		s = secret
		while s > 0:
			if s & 1 == 1:
				result ^= x & 1
			s >>= 1
			x >>= 1
		return result
	return f


def product(secret: int) -> Callable[[int], int]:
	"""
	This is our original attempt at a variation of Bernstein-Vazirani.

	Here, we replace XOR with AND.
	"""
	def f(x: int) -> int:
		result = 1
		s = secret
		while s > 0:
			if s & 1 == 1:
				result &= x & 1
			s >>= 1
			x >>= 1
		return result
	return f


def sum(secret: int) -> Callable[[int], int]:
	"""
	Unexplored possibility suggested by @Ashnah.

	Here, we will replace XOR with OR.
	"""
	def f(x: int) -> int:
		result = 0
		s = secret
		while s > 0:
			if s & 1 == 1:
				result |= x & 1
			s >>= 1
			x >>= 1
		return result
	return f


def svetlichny(string: int, secret: int, positions: list[int]) -> int:
	from functools import reduce
	from operator import mul, xor
	x = bits(string)
	s = bits(secret)
	products = [reduce(mul, subset, 1) for subset in subsets(x)]
	filtered_products = [products[j] * s[i] for (i, j) in enumerate(positions)]
	result = reduce(xor, filtered_products, 0)
	return result


## Generating truth tables

### Constants

The input to our function is an $m$-bit string.
Therefore there are $M = 2^m$ possible inputs.

Similarly, each function contains an $n$-bit secret string.
There are $N = 2^n$ possible secret strings.

In [23]:
# Number of bits.
n = 4

# Number of possible inputs: 2^n
N = (1 << n)

# The function we are evaluating.
f = svetlichny


### Filling out the truth table

Our truth table will be a matrix with $M$ rows and $N$ columns, representing each input and each secret string respectively.

In [24]:
def generate_matrix(positions: list[int]) -> Matrix:
	matrix = Matrix.new(N, N)
	for s in range(N):
		for x in range(N):
			matrix[x, s] = 1 if (f(x, s, positions) == 0) else -1
	return matrix


### Printing results

In [25]:
from itertools import combinations

# Set of possible positions where the bits of the secret string will be applied.
combos = list(combinations(range(N),  n))

for combo in combos:
	positions = list(combo)
	matrix = generate_matrix(positions)
	max_orthogonal_vectors = matrix.max_orthogonal_subset
	print("Positions:", combo)
	print("Function:", expression(positions))
	print("Orthogonal column indices:", max_orthogonal_vectors)
	print()


[[], [0], [1], [0, 1], [2], [0, 2], [1, 2], [0, 1, 2]]
Positions: (0, 1, 2)
Function: s₁0 ⊕ s₂𝑥₁ ⊕ s₃𝑥₂
Orthogonal column indices: [4, 5, 6, 7]

Positions: (0, 1, 4)
Function: s₁0 ⊕ s₂𝑥₁ ⊕ s₃𝑥₃
Orthogonal column indices: [4, 5, 6, 7]

Positions: (0, 2, 4)
Function: s₁0 ⊕ s₂𝑥₂ ⊕ s₃𝑥₃
Orthogonal column indices: [4, 5, 6, 7]

Positions: (1, 2, 3)
Function: s₁𝑥₁ ⊕ s₂𝑥₂ ⊕ s₃𝑥₁𝑥₂
Orthogonal column indices: [1, 3, 5, 7]

Positions: (1, 2, 4)
Function: s₁𝑥₁ ⊕ s₂𝑥₂ ⊕ s₃𝑥₃
Orthogonal column indices: [0, 1, 2, 3, 4, 5, 6, 7]

Positions: (1, 2, 5)
Function: s₁𝑥₁ ⊕ s₂𝑥₂ ⊕ s₃𝑥₁𝑥₃
Orthogonal column indices: [1, 3, 5, 7]

Positions: (1, 2, 6)
Function: s₁𝑥₁ ⊕ s₂𝑥₂ ⊕ s₃𝑥₂𝑥₃
Orthogonal column indices: [1, 3, 5, 7]

Positions: (1, 2, 7)
Function: s₁𝑥₁ ⊕ s₂𝑥₂ ⊕ s₃𝑥₁𝑥₂𝑥₃
Orthogonal column indices: [1, 3, 5, 7]

Positions: (1, 3, 4)
Function: s₁𝑥₁ ⊕ s₂𝑥₁𝑥₂ ⊕ s₃𝑥₃
Orthogonal column indices: [2, 3, 6, 7]

Positions: (1, 4, 5)
Function: s₁𝑥₁ ⊕ s₂𝑥₃ ⊕ s₃𝑥₁𝑥₃
Orthogonal column indices: [1, 3, 5, 7]

Positions: (